In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [5]:
pip install tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:010m
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install accelerate -U

In [3]:
# Install required libraries
!pip install transformers datasets pandas torch scikit-learn accelerate -U


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2Config, GPT2ForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import torch
from torch.utils.data import Dataset, DataLoader
from tf_keras.preprocessing.text import Tokenizer


In [7]:
df = pd.read_csv('/home/yadagiri/Downloads/finalfeatures_with_perplexity_burstness.csv')

In [8]:
df.head(2)

,Text,Label,Vocab Size,Avg Word Length,Density,active,passive,noun,pron,verb,...,det,propn,part,intj,punct,text_str,Flesch Reading Ease,Gunning Fog Index,Perplexity,Burstness
0,"['Basically there are many categories of "" Bes...",0,134,3.602230,0.498141,13,0,59,13,32,...,25,15,0,0,52,"['Basically there are many categories of "" Bes...",72.525622,10.477182,89.357222,0.860252
1,['salt is good for not dying in car crashes an...,0,216,4.014388,0.517986,17,5,93,18,67,...,28,8,2,3,54,['salt is good for not dying in car crashes an...,67.493766,10.975323,131.635424,1.227575


In [9]:

df = df.drop(columns=['text_str'])


In [10]:
df.head(2)

,Text,Label,Vocab Size,Avg Word Length,Density,active,passive,noun,pron,verb,...,adv,det,propn,part,intj,punct,Flesch Reading Ease,Gunning Fog Index,Perplexity,Burstness
0,"['Basically there are many categories of "" Bes...",0,134,3.602230,0.498141,13,0,59,13,32,...,23,25,15,0,0,52,72.525622,10.477182,89.357222,0.860252
1,['salt is good for not dying in car crashes an...,0,216,4.014388,0.517986,17,5,93,18,67,...,31,28,8,2,3,54,67.493766,10.975323,131.635424,1.227575


In [11]:
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.texts = dataframe['Text'].tolist()
        self.labels = dataframe['Label'].tolist()
        self.additional_features = dataframe.drop(columns=['Text', 'Label']).values.tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        additional_features = self.additional_features[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'additional_features': torch.tensor(additional_features, dtype=torch.float),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [12]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
# Create the datasets
train_dataset = TextDataset(train_df, tokenizer, max_len=512)
test_dataset = TextDataset(test_df, tokenizer, max_len=512)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [15]:
class GPT2ForTextClassification(GPT2ForSequenceClassification):
    def __init__(self, config, additional_feature_dim):
        super(GPT2ForTextClassification, self).__init__(config)
        self.additional_feature_dim = additional_feature_dim
        self.additional_features_fc = torch.nn.Linear(additional_feature_dim, config.hidden_size)
        self.classifier = torch.nn.Linear(config.hidden_size * 2, config.num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None, additional_features=None):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        hidden_state = outputs[0]  # Last hidden state

        # Take the mean of the hidden state to get a single vector representation of the input text
        pooled_output = hidden_state.mean(dim=1)

        # Process the additional features
        additional_features = self.additional_features_fc(additional_features)

        # Concatenate the text representation with the additional features
        combined_output = torch.cat((pooled_output, additional_features), dim=1)

        logits = self.classifier(combined_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        output = (loss,) + (logits,) + outputs[2:]
        return output

In [16]:
config = GPT2Config.from_pretrained('gpt2', num_labels=2)

In [17]:
model = GPT2ForTextClassification(config, additional_feature_dim=train_df.shape[1] - 2)

In [18]:
class PrintEpochCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"Starting epoch {state.epoch + 1}")

In [20]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Change the number of epochs here
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=250,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="steps"
)

In [21]:
from datasets import load_metric
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report

In [22]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = (preds == labels).mean()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [23]:
class PrintEpochCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"Starting epoch {state.epoch + 1}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[PrintEpochCallback()]
)


In [24]:
trainer.train()

Starting epoch 1


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.123400,0.303525,0.908110,0.895932,0.925163,0.910313
100,0.628800,0.233977,0.918080,0.933502,0.901713,0.917332
150,0.558800,0.392218,0.940282,0.922250,0.962684,0.942033
200,0.430900,0.352010,0.947065,0.951079,0.943515,0.947282
250,0.432200,1.475030,0.853736,0.776841,0.995922,0.872844
300,0.413800,0.390866,0.944187,0.907951,0.989600,0.947019
350,0.312400,0.586130,0.906260,0.845927,0.995310,0.914559
400,0.175100,0.303248,0.956419,0.926017,0.992863,0.958276
450,0.444000,0.275038,0.954363,0.922029,0.993475,0.956419
500,0.272400,0.175321,0.965875,0.983707,0.948002,0.965524


TrainOutput(global_step=4865, training_loss=0.1295346730414066, metrics={'train_runtime': 179705.4593, 'train_samples_per_second': 0.217, 'train_steps_per_second': 0.027, 'total_flos': 1.01705667764736e+16, 'train_loss': 0.1295346730414066, 'epoch': 1.0})

In [26]:
model_save_path = '/home/yadagiri/Downloads/SN BOSE TEAMS DATA/LAVANYA TEAM/GPT2'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /home/yadagiri/Downloads/SN BOSE TEAMS DATA/LAVANYA TEAM/GPT2


In [27]:
# Evaluate the model
eval_result = trainer.evaluate()

# Print evaluation metrics
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")
print(f"Precision: {eval_result['eval_precision']:.4f}")
print(f"Recall: {eval_result['eval_recall']:.4f}")
print(f"F1: {eval_result['eval_f1']:.4f}")

# Compute and print classification report
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids
print("\nClassification Report:\n", classification_report(labels, preds))

Accuracy: 0.9928
Precision: 0.9907
Recall: 0.9951
F1: 0.9929

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      4825
           1       0.99      1.00      0.99      4904

    accuracy                           0.99      9729
   macro avg       0.99      0.99      0.99      9729
weighted avg       0.99      0.99      0.99      9729

